In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import time
import gc
import pandas as pd
from tqdm import tqdm, tqdm_notebook, tnrange
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3"
config_tf = tf.ConfigProto()
config_tf.gpu_options.allow_growth = True
session = tf.Session(config=config_tf)

import keras
from keras.backend import reverse
from keras.initializers import glorot_normal,glorot_uniform, lecun_uniform,Orthogonal
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, SpatialDropout1D, Multiply, concatenate, Activation, Average
from keras.layers import Dropout, BatchNormalization, Concatenate, Conv1D, Flatten, Masking
from keras.layers import Bidirectional, CuDNNLSTM, CuDNNGRU, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import Sequence, to_categorical
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras import backend as K
from keras.engine.base_layer import Layer
from transformer import *
from keras.utils.training_utils import multi_gpu_model 

Using TensorFlow backend.


In [2]:
class Config(object):
    embeddingSize = 256
    sequenceLength = 90
    input_channels = 5
    istest = True

config = Config()

In [3]:
%%time
label_1 = pd.read_csv('./data/train_preliminary/user.csv')
label_2 = pd.read_csv('./data/train_semi_final/user.csv')
label = pd.concat([label_1, label_2], axis=0).reset_index(drop=True)
mats_train = []
mats_test = []
for col in tqdm(['creative_id', 'ad_id', 'advertiser_id', 'product_id', 'industry']):    
    mats_train.append(np.load('./inputs/{}_inputs_train.npy'.format(col)))
    mats_test.append(np.load('./inputs/{}_inputs_test.npy'.format(col)))

100%|██████████| 5/5 [00:04<00:00,  1.24it/s]

CPU times: user 396 ms, sys: 4.22 s, total: 4.62 s
Wall time: 4.6 s


In [4]:
class DataSequence(Sequence):
    def __init__(self, xs, y, batch_size=128, shuffle=True):
        self.xs = xs
        self.y = y
        self.batch_size = batch_size
        self.size = xs[0].shape[0]
        self.shuffle = shuffle
        if self.shuffle:
            state = np.random.get_state()
            for x in self.xs:
                np.random.set_state(state)
                np.random.shuffle(x)
            np.random.set_state(state)
            np.random.shuffle(self.y)
         
    def __len__(self):
        return int(np.ceil(self.size / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_idx = np.arange(idx * self.batch_size, min((idx + 1) * self.batch_size, self.size))
        batch_xs = [x[batch_idx] for x in self.xs]
        batch_y = self.y[batch_idx]
        # shuffle
        if self.shuffle:
            x = []
            for i in range(len(batch_xs)):
                x.append(batch_xs[i].copy())
            for i in range(len(x[0])):
                p = np.random.rand()
                if p < 0.8:
                    state = np.random.get_state()
                    for j in range(len(batch_xs)):
                        np.random.set_state(state)
                        np.random.shuffle(x[j][i])
            batch_xs = x        
        return batch_xs, batch_y

In [5]:
class Checkpoint(ModelCheckpoint):
    
    def __init__(self, model_saved, filepath, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1):
        ModelCheckpoint.__init__(self, filepath, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1)
        self.model = model_saved
        
    def set_model(self, model):
        pass

In [6]:
class Inception():
    def __init__(self, use_relu=True, use_norm=True):
        self.use_relu = use_relu
        self.use_norm = use_norm

    def __call__(self, input_x):
        self.branch1 = Sequential()
        self.branch1.add(Conv1D(filters=128, kernel_size=1, strides=1))
        
        self.branch2 = Sequential()
        self.branch2.add(Conv1D(filters=128, kernel_size=2, strides=1))
        self.branch2.add(BatchNormalization())
        self.branch2.add(ReLU())
        self.branch2.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same'))
        
        self.branch3 = Sequential()
        self.branch3.add(Conv1D(filters=128, kernel_size=3, strides=1))
        self.branch3.add(BatchNormalization())
        self.branch3.add(ReLU())
        self.branch3.add(Conv1D(filters=128, kernel_size=5, strides=1, padding='same'))
        
        self.branch4 = Sequential()
        self.branch4.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same'))
        
#         self.branch5 = Sequential()
#         self.branch5.add(Conv1D(filters=128, kernel_size=2, strides=1))
        
#         self.branch6 = Sequential()
#         self.branch6.add(Conv1D(filters=128, kernel_size=2, strides=1))
#         self.branch6.add(BatchNormalization())
#         self.branch6.add(ReLU())
#         self.branch6.add(Conv1D(filters=128, kernel_size=7, strides=1, padding='same'))
        
        branch1 = self.branch1(input_x)
        branch2 = self.branch2(input_x)
        branch3 = self.branch3(input_x)
        branch4 = self.branch4(input_x)
#         branch5 = self.branch5(input_x)
#         branch6 = self.branch6(input_x)
        
        ret = Concatenate(axis=1)([branch1, branch2, branch3, branch4])
        ret = BatchNormalization()(ret)
        ret = ReLU()(ret)
        return ret

In [7]:
def LSTM(config, n_cls=10):
    cols = ['creative_id', 'ad_id', 'advertiser_id', 'product_id', 'industry']
    n_in = len(cols)
    inputs = []
    outputs = []
    max_len = []
    for i in range(n_in):
        We = np.load('./w2v_256_10/{}_embedding_weight.npy'.format(cols[i]))
        We = np.vstack([We, np.zeros(config.embeddingSize)])
        inp = Input(shape=(config.sequenceLength,), dtype="int32")
        x = Embedding(We.shape[0], We.shape[1], weights=[We], trainable=False)(inp)
        inputs.append(inp)
        outputs.append(x)
        del We
        gc.collect()

    embedding_model = Model(inputs, outputs)        
    
    inputs = []
    for i in range(n_in):
        inp = Input(shape=(config.sequenceLength, config.embeddingSize, ))
        inputs.append(inp)
        
    all_input = Concatenate()(inputs)
    all_input = SpatialDropout1D(0.2)(all_input)
    lstm1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(all_input)
    lstm2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(lstm1)
    inc1 = Inception()(lstm1)
    inc2 = Inception()(lstm2)
    pool_1 = GlobalMaxPooling1D()(lstm1)
    pool_2 = GlobalMaxPooling1D()(lstm2)
    pool_3 = GlobalMaxPooling1D()(inc1)
    pool_4 = GlobalMaxPooling1D()(inc2)
    pool = Concatenate()([pool_1,pool_3, pool_2, pool_4])
    pool = Dropout(0.2)(pool)
    
    outputs = Dense(n_cls, activation='softmax')(pool)
    lstm_model = Model(inputs, outputs)
    model = Model(embedding_model.inputs, lstm_model(embedding_model.outputs))

    return model, lstm_model

In [8]:
def train_eval(config, fold, n_cls, xs_trn, y_trn, xs_val, y_val, xs_test=None, batch_size=256, logger=None):
    tf.reset_default_graph()
    K.clear_session()
    trn_generator = DataSequence(xs_trn, y_trn, batch_size=batch_size, shuffle=True)
    val_generator = DataSequence(xs_val, y_val, batch_size=batch_size, shuffle=False)
    if config.istest:
        test_generator = DataSequence(xs_test, np.zeros(xs_test[0].shape[0]), batch_size=batch_size, shuffle=False)
    model, lstm_model = LSTM(config, n_cls=n_cls)
    model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    ckp_cb = Checkpoint(lstm_model, './model/fold_{}_lstm_age_weight1.pkl'.format(fold), save_best_only=True, save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, min_lr=0, verbose=1)
    csvlogger = CSVLogger('./foldfoldfold_{}_logger.csv'.format(fold))
    es_cb = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=0, mode='auto')
    history = model.fit_generator(generator=trn_generator, 
                                epochs=15, 
                                verbose=1, 
                                workers=8,
                                use_multiprocessing=True,
                                max_q_size=48,
                                validation_data=val_generator,
                                callbacks=[ckp_cb, reduce_lr,csvlogger])
    lstm_model.load_weights('./model/fold_{}_lstm_age_weight.pkl'.format(fold))
    y_pred = model.predict_generator(val_generator)
    if config.istest:
        print("test set predicting")
        y_test = model.predict_generator(test_generator)
    del model
    if config.istest:
        return y_pred, y_test
    else:
        return y_pred

In [9]:
tmp_data = pd.read_csv('./data/train_preliminary/user.csv')

batch_size = 2048
y = label['age'].values
yc = to_categorical(y-1)
num_classes = 10
best_score = 0
oof = []
cv_pred_stack = np.zeros((mats_train[0].shape[0],num_classes))
if config.istest:
    test_pred_stack = np.zeros((mats_test[0].shape[0],num_classes))
kfold = StratifiedKFold(n_splits=5,random_state=2020)
for index, (idx_trn, idx_val) in tqdm(enumerate(kfold.split(tmp_data, tmp_data['age']))):  
    if index == 2:
        print("Fold %d"  % index)
        all_idx = np.arange(len(mats_train[0]))
        subtraction = list((set(all_idx).difference(set(idx_val))))
        xs_trn = [x[subtraction] for x in mats_train]
        xs_val = [x[idx_val] for x in mats_train]
        print(len(xs_trn[0]), len(xs_val[0]))
        y_trn, y_val = yc[subtraction], yc[idx_val]
    #     del mats_train
    #     gc.collect()
        if config.istest:
            y_pred, y_test = train_eval(config, index, num_classes, xs_trn, y_trn, xs_val, y_val, mats_test, batch_size) 
        else:
            y_pred = train_eval(config, index, num_classes, xs_trn, y_trn, xs_val, y_val, mats_test, batch_size) 

        cv_pred_stack[idx_val] = y_pred
        if config.istest:
            print("result saved")
            test_pred_stack += y_test / 5

acc = accuracy_score(y, np.argmax(cv_pred_stack,axis=1)+1)
print('score: ', acc)

1it [00:00,  6.08it/s]

Fold 2
2820000 180000





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/15
1377/1377 [==============================] - 269s 196ms/step - loss: 1.3975 - acc: 0.4267 - val_loss: 1.2745 - val_acc: 0.4734
Epoch 2/15
1377/1377 [==============================] - 242s 175ms/step - loss: 1.2829 - acc: 0.4693 - val_loss: 1.2396 - val_acc: 0.4857
Epoch 3/15
1377/1377 [==============================] - 239s 173ms/step - loss: 1.2527 - acc: 0.4805 - val_loss: 1.2201 - val_acc: 0.4935
Epoch 4/15
1377/1377 [==============================] - 239s 173ms/step - loss: 1.2351 - acc: 0.4874 - val_loss: 1.2077 - val_acc: 0.4988
Epoch 5/15
1377/1377 [==============================] - 239s 174ms/step - loss: 1.2224 - acc: 0.4921 - val_loss: 1.2021 - val_acc: 0.5002
Epoch 6/15
1377/1377 [==============================] - 242s 176ms/step -

2it [1:02:09, 1864.54s/it]


ValueError: You are trying to load a weight file containing 3 layers into a model with 13 layers.

In [9]:
np.save('./fold0_test_pred.npy', test_pred_stack)
np.save('./fold0_val_pred.npy', cv_pred_stack)